In [1]:
from pricing_library.market import MarketData
from pricing_library.option import Option
from pricing_library.trinomial_tree import TrinomialTree

from datetime import datetime

In [2]:
mkt = MarketData(
    interest_rate=0.04,
    volatility=0.2,
    spot_price=100,
    dividend_price=3,
    dividend_ex_date=datetime(2020, 6, 1),
)
opt = Option(
    option_type="call",
    exercise_type="eu",
    strike_price=100,
    maturity_date=datetime(2021, 1, 1),
)

t = TrinomialTree(mkt, opt, datetime(2020, 1, 1), 100)

RecursionError: maximum recursion depth exceeded while calling a Python object